In [2]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
# Load dataset 
train = pd.read_csv('remove_duplicates_transform_label_address_train.csv')
test = pd.read_csv('remove_duplicates_transform_label_address_test.csv')
sample = pd.read_csv('input/sampleSubmission.csv')
train['Zipcode']=train['Zipcode'].astype(str)
test['Zipcode']=test['Zipcode'].astype(str)

weather = pd.read_csv('input/weather.csv')
# spray = pd.read_csv('input/spray.csv')
# weather station coordinates
station1 = [-87.933 , 41.995]
station2 = [-87.752 , 41.786]

In [56]:
def generate_date_features(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].apply(lambda x: x.year)
    df['Month'] = df['Date'].apply(lambda x: x.month)
    df['DayOfMonth'] = df['Date'].apply(lambda x: x.day)
    # df['DayOfWeekName'] = df['Date'].apply(lambda x: x.day_name())
    df['DayOfWeek'] = df['Date'].apply(lambda x: x.dayofweek)
    df['DayOfYear'] = df['Date'].apply(lambda x: x.dayofyear)
#     df['WeekOfYear'] = df['Date'].apply(lambda x: x.weekofyear)
#     df['IsLeapYear'] = df['Date'].apply(lambda x: x.is_leap_year)
#     df['IsLeapYear'] = df['IsLeapYear'].astype(str)
#     df['Quarter'] = df['Date'].apply(lambda x: x.quarter)

def transform_df(df):
    df_ = df.copy()
    df_['Lat_int'] = df_.Latitude.astype(int)
    df_['Long_int'] = df_.Longitude.astype(int)
    return df_

In [57]:
train2 = transform_df(train)
test2 = transform_df(test)

In [58]:
train2.dtypes 

Date                object
Species             object
Block                int64
Street              object
Trap                object
Latitude           float64
Longitude          float64
AddressAccuracy      int64
WnvPresent           int64
City                object
State               object
Zipcode             object
Lat_int              int64
Long_int             int64
dtype: object

# featurize weather, remove missing values

In [59]:
# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [71]:
# Merge with weather data
train3 = train2.merge(weather, on='Date')
test3 = test2.merge(weather, on='Date')

In [72]:
train3.dtypes

Date                object
Species             object
Block                int64
Street              object
Trap                object
Latitude           float64
Longitude          float64
AddressAccuracy      int64
WnvPresent           int64
City                object
State               object
Zipcode             object
Lat_int              int64
Long_int             int64
Tmax_x               int64
Tmin_x               int64
Tavg_x              object
Depart_x            object
DewPoint_x           int64
WetBulb_x           object
Heat_x              object
Cool_x              object
Sunrise_x           object
Sunset_x            object
Depth_x             object
Water1_x             int64
SnowFall_x          object
PrecipTotal_x       object
StnPressure_x       object
SeaLevel_x          object
ResultSpeed_x      float64
ResultDir_x          int64
AvgSpeed_x          object
Tmax_y               int64
Tmin_y               int64
Tavg_y              object
Depart_y             int64
D

In [73]:
# drop address columns
train3 = train3.drop(['Date','WnvPresent'],  axis = 1)
test3 = test3.drop(['Date','Id'],axis = 1)

In [75]:
train3.shape

(8475, 50)

In [69]:
test3.shape

(116293, 50)

# label encoder

In [76]:
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [77]:
def get_label_encoder(train, test, cols):
    lbl = preprocessing.LabelEncoder()
    for col in cols:
        lbl.fit(list(train[col].values) + list(test[col].values))
        train[col] = lbl.transform(train[col].values)
        test[col] = lbl.transform(test[col].values)
def convert_string_to_num(df, str_columns):
    for col in str_columns:
        df[col] = df[col].astype(float)
def get_metrics(y_true, y_pred):
    print("Precision: %1.3f" % precision_score(y_true, y_pred))
    print("Recall: %1.3f" % recall_score(y_true, y_pred))
    print("F1: %1.3f" % f1_score(y_true, y_pred))
    print("AUC: %1.3f" % roc_auc_score(y_true, y_pred))

In [78]:
all_obj_cols = train3.select_dtypes(include=['object']).columns
all_obj_cols

Index(['Species', 'Street', 'Trap', 'City', 'State', 'Zipcode', 'Tavg_x',
       'Depart_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x',
       'Depth_x', 'SnowFall_x', 'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x',
       'AvgSpeed_x', 'Tavg_y', 'WetBulb_y', 'Heat_y', 'Cool_y',
       'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y', 'AvgSpeed_y'],
      dtype='object')

In [79]:
str_columns = [u'Tavg_x', u'Depart_x', u'WetBulb_x',
       u'Heat_x', u'Cool_x', u'Sunrise_x', u'Sunset_x', u'Depth_x',
       u'SnowFall_x', u'PrecipTotal_x', u'StnPressure_x', u'SeaLevel_x',
       u'AvgSpeed_x', u'Tavg_y', u'WetBulb_y', u'Heat_y', u'Cool_y',
       u'PrecipTotal_y', u'StnPressure_y', u'SeaLevel_y', u'AvgSpeed_y']
convert_string_to_num(train3,str_columns )
convert_string_to_num(test3,str_columns )

In [91]:
obj_cols = [u'Species', u'Street', u'Trap','State','City','Zipcode']
get_label_encoder(train3, test3, obj_cols)

In [95]:
# remove missing data
train4 = train3.ix[:,(train3 != -1).any(axis=0)]
test4 = test3.ix[:,(test3 != -1).any(axis=0)]

/Users/juyang/anaconda2/envs/sparkbeyond/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/juyang/anaconda2/envs/sparkbeyond/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [96]:
test4.shape

(116293, 43)

In [97]:
train4.head()

,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,City,State,Zipcode,...,DewPoint_y,WetBulb_y,Heat_y,Cool_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y
0,2,41,36,1,41.954690,-87.800991,9,2,0,27,...,59,66.0,0.0,12.0,0.0,29.44,30.09,5.8,16,7.4
1,3,41,36,1,41.954690,-87.800991,9,2,0,27,...,59,66.0,0.0,12.0,0.0,29.44,30.09,5.8,16,7.4
2,3,62,30,8,41.994991,-87.769279,9,2,0,37,...,59,66.0,0.0,12.0,0.0,29.44,30.09,5.8,16,7.4
3,2,79,120,15,41.974089,-87.824812,8,2,0,44,...,59,66.0,0.0,12.0,0.0,29.44,30.09,5.8,16,7.4
4,3,79,120,15,41.974089,-87.824812,8,2,0,44,...,59,66.0,0.0,12.0,0.0,29.44,30.09,5.8,16,7.4


In [98]:
X = train4
y = train.WnvPresent.values
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3,
                                                    random_state=42) # need to convert to np array

In [107]:
gbt = ensemble.GradientBoostingClassifier()

learning_rate = [0.01, 0.02, 0.05, 0.1]
max_depth = [2, 3, 4, 6, 8]
n_estimators = [20, 100, 150, 200]

tuned_parameters = [{'max_depth': max_depth,
                     'n_estimators':n_estimators,
                     'learning_rate':learning_rate 
                    }]
n_folds = 3

# cv with aucroc
clf = GridSearchCV(gbt, 
                   tuned_parameters, 
                   cv=n_folds, 
                   refit=True, 
                   scoring='roc_auc') # what if i set recall

In [108]:
%%time
clf.fit(X_train, y_train)

CPU times: user 5min 41s, sys: 1.03 s, total: 5min 42s
Wall time: 5min 44s


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [2, 3, 4, 6, 8], 'n_estimators': [20, 100, 150, 200], 'learning_rate': [0.01, 0.02, 0.05, 0.1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [109]:
clf.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}

In [110]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [111]:
get_metrics(y_train, y_train_pred)

Precision: 0.769
Recall: 0.064
F1: 0.118
AUC: 0.531


In [115]:
get_metrics(y_test, y_test_pred)

Precision: 0.250
Recall: 0.014
F1: 0.026
AUC: 0.506


In [116]:
def create_prediction(clf, test, sample):
    predictions = clf.predict(test)
    sample['WnvPresent'] = predictions 
    # wait why is it not binary prediction, all zeros????
    # why it is still all 0
    print (sample.WnvPresent.value_counts())

In [117]:
create_prediction(clf, test4, sample)

0    116171
1       122
Name: WnvPresent, dtype: int64


In [ ]:
sample.to_csv('exp3.csv', index=False)
# change naming system